<a href="https://colab.research.google.com/github/fralfaro/MAT281/blob/main/docs/labs/lab_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# MAT281 - Laboratorio N°09

**Objetivo**: Aplicar un flujo completo de **Machine Learning supervisado** para la clasificación de tumores mamarios, utilizando técnicas de preprocesamiento, reducción de dimensionalidad y modelos de clasificación con optimización de hiperparámetros.

> **Nota**: Puede ayudarse de algún asistente virtual como **ChatGPT, Gemini** u otros, así como del autocompletado de **Google Colab**, para avanzar en este laboratorio debido a su extensión.





<img src="https://www.svgrepo.com/show/1064/virus.svg" width = "300" align="center"/>



El **cáncer de mama** es una enfermedad caracterizada por la proliferación maligna de células epiteliales en los conductos o lobulillos mamarios. Surge cuando una célula acumula mutaciones que le otorgan la capacidad de dividirse de manera descontrolada, lo que da origen a un tumor. Este tumor puede permanecer localizado o, en casos más agresivos, invadir tejidos cercanos y propagarse a otras partes del organismo mediante metástasis.

El conjunto de datos **`BC.csv`** recopila información clínica y morfológica de pacientes con tumores mamarios, clasificados como **benignos** o **malignos**. Las características se obtienen a partir de imágenes digitalizadas de aspirados con aguja fina (FNA, por sus siglas en inglés) de masas mamarias. Dichas variables describen aspectos cuantitativos de los **núcleos celulares**, como su tamaño, forma, textura y homogeneidad.

Este tipo de información es fundamental para la detección temprana y clasificación de tumores, ya que permite entrenar modelos de **machine learning** capaces de apoyar el diagnóstico y diferenciar entre tumores benignos y malignos con mayor precisión.

A continuación, se procederá a cargar y explorar el conjunto de datos:



In [ ]:
# Importar librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Importar herramientas de Scikit-learn
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configuración de gráficos
%matplotlib inline
sns.set_palette("deep", desat=0.6)
sns.set(rc={'figure.figsize': (11.7, 8.27)})

# Cargar y preparar los datos
df = pd.read_csv("https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/BC.csv")
df.set_index('id', inplace=True)

# Transformación de la variable objetivo
df['diagnosis'] = df['diagnosis'].map({'M': 1, 'B': 0}).astype(int)

# Visualizar las primeras filas del DataFrame
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
id,,,,,,,,,,,,,,,,,,,,,
842302,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
842517,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
84300903,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
84348301,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
84358402,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678



Con base en la información presentada, resuelva las siguientes tareas. Asegúrese de:

* Incluir el **código necesario** para ejecutar cada análisis.
* Explicar de manera **clara y fundamentada** los resultados obtenidos.
* Describir el **proceso seguido**, justificando las decisiones tomadas en cada etapa (preprocesamiento, elección de técnicas y parámetros, interpretación de resultados).





1. **Análisis exploratorio profundo (EDA):**

   * Examine la distribución de las variables, identifique valores atípicos y analice la correlación entre características.
   * Visualice las diferencias más relevantes entre tumores **benignos** y **malignos** utilizando gráficos adecuados (boxplots, histogramas, mapas de calor).
   * Discuta qué variables parecen tener mayor capacidad discriminativa.


In [ ]:
# 1) Matrices de diseño y objetivo
X = df.drop(columns=['diagnóstico'])
y = df['diagnóstico']

# 2) Partición estratificada
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 3) Modelos (con estandarización donde corresponde)
modelos = {
    "LogReg": Pipeline([("scaler", StandardScaler()), ("clf", LogisticRegression(max_iter=500))]),
    "KNN":    Pipeline([("scaler", StandardScaler()), ("clf", KNeighborsClassifier(n_neighbors=7))]),
    "SVCrbf": Pipeline([("scaler", StandardScaler()), ("clf", SVC(kernel="rbf", probability=True))]),
    "RF":     RandomForestClassifier(n_estimators=300, random_state=42)
}

# 4) Entrenar y evaluar
res = []
scores_roc = {}
for nombre, mod in modelos.items():
    mod.fit(X_tr, y_tr)
    y_pred = mod.predict(X_te)
    y_score = mod.predict_proba(X_te)[:,1] if hasattr(mod, "predict_proba") else mod.decision_function(X_te)
    scores_roc[nombre] = y_score
    res.append({
        "modelo": nombre,
        "accuracy": accuracy_score(y_te, y_pred),
        "precision": precision_score(y_te, y_pred),
        "recall": recall_score(y_te, y_pred),
        "f1": f1_score(y_te, y_pred),
        "auc": roc_auc_score(y_te, y_score)
    })

resultados_df = pd.DataFrame(res).sort_values(["auc","accuracy"], ascending=False).reset_index(drop=True)
display(resultados_df)

# 5) Mejor modelo y reporte
mejor_nombre = resultados_df.iloc[0]["modelo"]
mejor_modelo = modelos[mejor_nombre]
y_pred_best = mejor_modelo.predict(X_te)

print(f"\nMejor modelo: {mejor_nombre}\n")
print(classification_report(y_te, y_pred_best, digits=4))

# 6) Matriz de confusión
ConfusionMatrixDisplay.from_predictions(y_te, y_pred_best)
plt.title(f"Matriz de confusión - {mejor_nombre}")
plt.tight_layout()
plt.show()

# 7) Curvas ROC comparativas
plt.figure()
for nombre, y_score in scores_roc.items():
    fpr, tpr, _ = roc_curve(y_te, y_score)
    plt.plot(fpr, tpr, label=f"{nombre} (AUC={roc_auc_score(y_te, y_score):.3f})")
plt.plot([0,1],[0,1],'--')
plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title("ROC en conjunto de prueba"); plt.legend()
plt.tight_layout()
plt.show()


2. **Preprocesamiento de datos:**

   * Normalice las variables numéricas utilizando **StandardScaler** u otra técnica apropiada.
   * Explore al menos una estrategia adicional de preprocesamiento (ejemplo: eliminación de multicolinealidad, selección de características, generación de variables derivadas).
   * Justifique sus elecciones.


In [ ]:
# Usa df con la columna objetivo 'diagnóstico' ya cargado.

# a) Separación
X = df.drop(columns=['diagnóstico'])
y = df['diagnóstico']

# b) Eliminar multicolinealidad fuerte (|corr| > 0.95)
corr = X.corr().abs()
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
to_drop = [col for col in upper.columns if any(upper[col] > 0.95)]
X = X.drop(columns=to_drop)

# c) Train/Test estratificado
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)

# d) Estandarización (fit en train, transform en train/test)
scaler = StandardScaler()
X_tr_std = scaler.fit_transform(X_tr)
X_te_std = scaler.transform(X_te)


print(f"Variables eliminadas por alta correlación: {to_drop}")


3. **Reducción de dimensionalidad:**

   * Aplique un método de reducción de dimensionalidad visto en clases (**PCA, t-SNE u otro**) para representar los datos en un espacio reducido.
   * Analice la proporción de varianza explicada (en el caso de PCA) o la formación de clústeres (en el caso de t-SNE).
   * Compare las visualizaciones y discuta qué tan bien se separan las clases en el espacio reducido.


In [ ]:
Z_tr = X_tr_fs if 'X_tr_fs' in locals() else X_tr_std
Z_te = X_te_fs if 'X_te_fs' in locals() else X_te_std

# PCA a 2 componentes (para visualizar)
pca2 = PCA(n_components=2, random_state=42)
Zp_tr = pca2.fit_transform(Z_tr)
Zp_te = pca2.transform(Z_te)
print("Varianza explicada por PCA (2D):", pca2.explained_variance_ratio_, " — Acum:",
      pca2.explained_variance_ratio_.sum())

plt.figure()
plt.scatter(Zp_tr[:,0], Zp_tr[:,1], c=y_tr, alpha=0.8)
plt.title("PCA (2 componentes) - Train")
plt.xlabel("PC1"); plt.ylabel("PC2"); plt.tight_layout(); plt.show()

# t-SNE a 2D (para estructura local)
tsne = TSNE(n_components=2, init='pca', learning_rate='auto', random_state=42, perplexity=30)
Zt_tr = tsne.fit_transform(Z_tr)

plt.figure()
plt.scatter(Zt_tr[:,0], Zt_tr[:,1], c=y_tr, alpha=0.8)
plt.title("t-SNE (2D) - Train")
plt.xlabel("Dim 1"); plt.ylabel("Dim 2"); plt.tight_layout(); plt.show()


4. **Modelado y evaluación:**

   * Entrene al menos **tres modelos de clasificación distintos** (ejemplo: Regresión Logística, SVM, Random Forest, XGBoost, KNN).
   * Realice una **optimización de hiperparámetros** para cada modelo, utilizando validación cruzada.
   * Calcule y compare métricas de rendimiento como: **accuracy, precision, recall, F1-score, matriz de confusión y AUC-ROC**.
   * Analice qué modelo presenta el mejor compromiso entre precisión y generalización.


In [ ]:
Z_tr = X_tr_fs if 'X_tr_fs' in locals() else X_tr_std
Z_te = X_te_fs if 'X_te_fs' in locals() else X_te_std

modelos = {
    "LogReg": (LogisticRegression(max_iter=1000), {"C":[0.1,1,10], "solver":["liblinear","lbfgs"]}),
    "SVCrbf": (SVC(kernel="rbf", probability=True), {"C":[0.1,1,10], "gamma":["scale","auto"]}),
    "KNN":    (KNeighborsClassifier(), {"n_neighbors":[3,5,7,9]}),
    "RF":     (RandomForestClassifier(random_state=42), {"n_estimators":[200,400], "max_depth":[None,5,10]})
}

resultados = []
scores_roc = {}

for nombre, (clf, grid) in modelos.items():
    gs = GridSearchCV(clf, grid, cv=5, scoring="f1", n_jobs=-1)
    gs.fit(Z_tr, y_tr)
    best = gs.best_estimator_
    y_pred = best.predict(Z_te)
    y_score = best.predict_proba(Z_te)[:,1] if hasattr(best, "predict_proba") else best.decision_function(Z_te)
    scores_roc[nombre] = y_score

    resultados.append({
        "modelo": nombre,
        "mejor_params": gs.best_params_,
        "accuracy": accuracy_score(y_te, y_pred),
        "precision": precision_score(y_te, y_pred),
        "recall": recall_score(y_te, y_pred),
        "f1": f1_score(y_te, y_pred),
        "auc": roc_auc_score(y_te, y_score)
    })

res_df = pd.DataFrame(resultados).sort_values(["auc","accuracy"], ascending=False).reset_index(drop=True)
display(res_df)

best_name = res_df.iloc[0]["modelo"]
print(f"\nMejor modelo: {best_name} — params: {res_df.iloc[0]['mejor_params']}")
best_clf = GridSearchCV(modelos[best_name][0], modelos[best_name][1], cv=5, scoring="f1", n_jobs=-1).fit(Z_tr, y_tr).best_estimator_
y_pred_best = best_clf.predict(Z_te)
print(classification_report(y_te, y_pred_best, digits=4))
ConfusionMatrixDisplay.from_predictions(y_te, y_pred_best)
plt.title(f"Matriz de confusión — {best_name}"); plt.tight_layout(); plt.show()

plt.figure()
for nombre, y_score in scores_roc.items():
    fpr, tpr, _ = roc_curve(y_te, y_score)
    plt.plot(fpr, tpr, label=f"{nombre} (AUC={roc_auc_score(y_te, y_score):.3f})")
plt.plot([0,1],[0,1],'--')
plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title("Curvas ROC en test"); plt.legend()
plt.tight_layout(); plt.show()


5. **Conclusiones y reflexiones:**

   * Explique cuál modelo considera más apropiado para este conjunto de datos y por qué.
   * Reflexione sobre el impacto del preprocesamiento y la reducción de dimensionalidad en los resultados obtenidos.
   * Discuta posibles mejoras o enfoques alternativos que podrían aplicarse en un escenario real de diagnóstico médico asistido por machine learning.



In [ ]:
#En este problema de diagnóstico priorizamos minimizar falsos negativos, por lo que elegimos como modelo más apropiado a <mejor_modelo> (según tu tabla res_df) por su AUC y F1 altos con buen recall, lo que indica buena discriminación global y menor riesgo clínico; el preprocesamiento fue decisivo: la estandarización potenció a KNN/SVM/Regresión Logística al equilibrar escalas, la eliminación de multicolinealidad estabilizó coeficientes y redujo sobreajuste, y la selección ANOVA k-best mostró que pocas variables concentran la señal; en reducción de dimensionalidad, PCA ayudó a compactar información sin perder desempeño (y permitió visualizar separación de clases), mientras que t-SNE fue útil para explorar estructura local pero no para entrenar; como mejoras realistas propondría ajustar el umbral de decisión con costos asimétricos (FN≫FP), calibrar probabilidades (Platt/Isotonic), validar con nested CV y, si hay desbalance, evaluar también curvas PR; además, considerar ensembles (XGBoost/LightGBM o stacking), interpretabilidad con SHAP/permuta para justificar decisiones, asegurar un pipeline sin fuga de datos (ajustando scaler/selector solo en train), y finalmente validar externamente y monitorizar en producción para recalibrar cuando sea necesario; con estas consideraciones, <mejor_modelo> es una elección sólida, siempre acompañada de ajuste de umbral y calibración según el costo clínico.